In [1]:
# ===============================================
# 라이브러리 설치
# ===============================================
# 필요한 패키지 설치 (최초 1회만 실행)
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-openai
!pip install -q pypdf
!pip install -q pymupdf
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q sentence-transformers
!pip install -q rank_bm25


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ===============================================
# 셀 3: 라이브러리 임포트
# ===============================================
import os
import re
import json
import time
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime

# LangChain 핵심 모듈
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
)
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever

import warnings

warnings.filterwarnings("ignore")

print("모든 라이브러리 임포트 완료")

모든 라이브러리 임포트 완료


In [ ]:
# ===============================================
# 셀 4: 경로 설정
# ===============================================
# 파일 경로 설정
PDF_PATH = r"D:\data"  # PDF 파일이 있는 폴더
PERSIST_DIRECTORY = r"D:\data\chroma_db"  # 벡터 DB 저장 경로

print(f"PDF 경로: {PDF_PATH}")
print(f"벡터 DB 저장 경로: {PERSIST_DIRECTORY}")

PDF 경로: D:\data
벡터 DB 저장 경로: D:\data\chroma_db


In [4]:
# ===============================================
# 셀 5: 모든 함수 정의
# ===============================================


def load_pdf_documents_improved(directory_path: str) -> List[Document]:
    """
    한국어 의료 문서에 최적화된 PDF 로딩
    """
    print(f"PDF 파일 로드 중: {directory_path}")

    pdf_files = list(Path(directory_path).glob("*.pdf"))

    if not pdf_files:
        print("PDF 파일을 찾을 수 없습니다!")
        return []

    all_docs = []

    for pdf_file in pdf_files[:2]:  # 2개만 로드
        print(f"  로딩 중: {pdf_file.name}")

        # PyMuPDF 로더 사용 (한국어 처리 개선)
        loader = PyMuPDFLoader(str(pdf_file))
        docs = loader.load()

        # 문서 전처리
        for doc in docs:
            # 메타데이터 추가
            doc.metadata["source"] = pdf_file.name
            doc.metadata["file_path"] = str(pdf_file)

            # 텍스트 정리
            content = doc.page_content
            content = re.sub(r"\s+", " ", content)
            content = re.sub(r"-\s*\d+\s*-", "", content)
            content = re.sub(r"\n\s*\n", "\n", content)

            doc.page_content = content.strip()

        all_docs.extend(docs)

    print(f"총 {len(all_docs)}개 페이지 로드 완료")
    return all_docs


def create_korean_optimized_chunks(
    documents: List[Document],
) -> Dict[str, List[Document]]:
    """
    한국어 의료 문서에 최적화된 여러 청킹 전략 비교
    """
    results = {}

    # 전략 1: 한국어 특화 RecursiveCharacterTextSplitter
    print("\n텍스트 분할 방법 1: 한국어 최적화 RecursiveCharacterTextSplitter")
    korean_separators = [
        "\n\n",  # 단락
        "\n",  # 줄바꿈
        "다.",  # 한국어 문장 종결
        "요.",
        "함.",
        ". ",  # 영문 문장
        ")",  # 번호 목록
        " ",  # 공백
        "",
    ]

    korean_recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=korean_separators,
        length_function=len,
    )
    results["korean_recursive"] = korean_recursive_splitter.split_documents(documents)
    print(f"  - 생성된 청크 수: {len(results['korean_recursive'])}")

    # 전략 2: 기본 RecursiveCharacterTextSplitter
    print("\n텍스트 분할 방법 2: 기본 RecursiveCharacterTextSplitter")
    basic_recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=100, length_function=len
    )
    results["basic_recursive"] = basic_recursive_splitter.split_documents(documents)
    print(f"  - 생성된 청크 수: {len(results['basic_recursive'])}")

    # 전략 3: CharacterTextSplitter
    print("\n텍스트 분할 방법 3: CharacterTextSplitter")
    character_splitter = CharacterTextSplitter(
        chunk_size=500, chunk_overlap=100, separator="\n"
    )
    results["character"] = character_splitter.split_documents(documents)
    print(f"  - 생성된 청크 수: {len(results['character'])}")

    return results


def add_medical_metadata(chunks: List[Document]) -> List[Document]:
    """
    의료 문서 청크에 섹션 메타데이터 추가
    """
    print("\n메타데이터 추가 중...")

    for chunk in chunks:
        content = chunk.page_content.lower()

        # 섹션 자동 분류
        if any(keyword in content for keyword in ["진단", "검사", "기준"]):
            chunk.metadata["section"] = "diagnosis"
            chunk.metadata["section_kr"] = "진단"
        elif any(keyword in content for keyword in ["운동", "신체활동", "활동"]):
            chunk.metadata["section"] = "exercise"
            chunk.metadata["section_kr"] = "운동요법"
        elif any(keyword in content for keyword in ["약물", "치료", "처방"]):
            chunk.metadata["section"] = "treatment"
            chunk.metadata["section_kr"] = "치료"
        elif any(keyword in content for keyword in ["합병증", "부작용"]):
            chunk.metadata["section"] = "complications"
            chunk.metadata["section_kr"] = "합병증"
        elif any(keyword in content for keyword in ["식이", "영양", "식사"]):
            chunk.metadata["section"] = "diet"
            chunk.metadata["section_kr"] = "식이요법"
        else:
            chunk.metadata["section"] = "general"
            chunk.metadata["section_kr"] = "일반"

    return chunks


def get_embedding_model(model_type: str = "korean"):
    """
    임베딩 모델 선택 (한국어/다국어/OpenAI)
    """
    if model_type == "korean":
        print("한국어 특화 임베딩 모델 로딩 중...")
        return HuggingFaceEmbeddings(
            model_name="jhgan/ko-sroberta-multitask",
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
    elif model_type == "multilingual":
        print("다국어 임베딩 모델 로딩 중...")
        return HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
            model_kwargs={"device": "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )
    else:  # openai
        print("OpenAI 임베딩 모델 사용")
        return OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=100)


def create_vectorstore_safe(
    chunks: List[Document], embeddings, persist_dir: str, batch_size: int = 50
):
    """
    안전한 벡터 DB 생성 (큰 문서도 처리 가능)
    """
    print(f"\n벡터 DB 생성 중 (총 {len(chunks)}개 청크)...")

    if len(chunks) > 200:
        print(f"청크가 많아 배치 처리를 사용합니다 (배치 크기: {batch_size})")

        vectorstore = None
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i : i + batch_size]
            batch_end = min(i + batch_size, len(chunks))
            print(f"  처리 중: {i+1}-{batch_end}/{len(chunks)}")

            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=batch,
                    embedding=embeddings,
                    persist_directory=persist_dir,
                    collection_metadata={"hnsw:space": "cosine"},
                )
            else:
                vectorstore.add_documents(batch)

            if i + batch_size < len(chunks):
                time.sleep(0.5)
    else:
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            persist_directory=persist_dir,
            collection_metadata={"hnsw:space": "cosine"},
        )

    vectorstore.persist()
    print("벡터 DB 저장 완료")
    return vectorstore


def interpret_chroma_score(distance):
    """
    Chroma 거리 점수를 유사도로 변환하고 해석

    중요: Chroma는 거리(distance)를 반환하므로 낮을수록 좋음!
    """
    similarity_percent = (1 - distance / 2) * 100 if distance < 2 else 0

    if distance < 0.3:
        quality = "매우 좋음"
    elif distance < 0.5:
        quality = "좋음"
    elif distance < 0.7:
        quality = "보통"
    elif distance < 1.0:
        quality = "낮음"
    else:
        quality = "매우 낮음"

    return f"""
    거리 점수: {distance:.3f}
    유사도: {similarity_percent:.1f}%
    품질: {quality}
    """


def evaluate_search_quality(vectorstore, test_queries: List[str]):
    """
    검색 품질 평가 및 점수 해석
    """
    print("\n" + "=" * 70)
    print("검색 품질 평가")
    print("=" * 70)

    results = []

    for query in test_queries:
        print(f"\n질문: {query}")
        print("-" * 50)

        # 벡터 검색
        vector_results = vectorstore.similarity_search_with_score(query, k=3)

        for i, (doc, score) in enumerate(vector_results, 1):
            print(f"\n  결과 {i}:")
            print(interpret_chroma_score(score))
            print(f"    섹션: {doc.metadata.get('section_kr', 'N/A')}")
            print(f"    내용: {doc.page_content[:150]}...")

        # MMR 검색
        print("\n  [MMR 검색 - 다양성 확보]")
        mmr_results = vectorstore.max_marginal_relevance_search(query, k=3)
        for i, doc in enumerate(mmr_results, 1):
            print(
                f"    {i}. [{doc.metadata.get('section_kr', 'N/A')}] {doc.page_content[:80]}..."
            )

        results.append(
            {
                "query": query,
                "top_score": vector_results[0][1] if vector_results else None,
            }
        )

    return results


print("모든 함수 정의 완료")

모든 함수 정의 완료


In [5]:
# ===============================================
# 셀 6: Step 1 - PDF 문서 로드
# ===============================================
print("\n[Step 1] PDF 문서 로드")
documents = load_pdf_documents_improved(PDF_PATH)

if documents:
    print(f"\n로드 완료:")
    print(f"  - 총 페이지 수: {len(documents)}")
    print(f"  - 첫 페이지 미리보기:")
    print(f"    {documents[0].page_content[:300]}...")


[Step 1] PDF 문서 로드
PDF 파일 로드 중: D:\data
  로딩 중: 2023 당뇨병 진료지침_전문_240620.pdf
총 428개 페이지 로드 완료

로드 완료:
  - 총 페이지 수: 428
  - 첫 페이지 미리보기:
    1 Clinical Practice Guidelines for Diabetes...


In [6]:
# ===============================================
# 셀 7: Step 2 - 테스트 쿼리 정의
# ===============================================
print("\n[Step 2] 테스트 쿼리 정의")
test_queries = [
    "이 문서의 주요 내용은 무엇인가요?",
    "당뇨병의 진단기준에 대해 알려주세요.",
    "당뇨병의 운동요법은 어떤것이 있나요?",
]

for i, query in enumerate(test_queries, 1):
    print(f"  {i}. {query}")


[Step 2] 테스트 쿼리 정의
  1. 이 문서의 주요 내용은 무엇인가요?
  2. 당뇨병의 진단기준에 대해 알려주세요.
  3. 당뇨병의 운동요법은 어떤것이 있나요?


In [7]:
# ===============================================
# 셀 8: Step 3 - 임베딩 모델 로드
# ===============================================
print("\n[Step 3] 임베딩 모델 로드")
EMBEDDING_TYPE = "korean"  # korean / multilingual / openai
embeddings = get_embedding_model(EMBEDDING_TYPE)
print(f"임베딩 모델 로드 완료: {EMBEDDING_TYPE}")


[Step 3] 임베딩 모델 로드
한국어 특화 임베딩 모델 로딩 중...
임베딩 모델 로드 완료: korean


In [8]:
# ===============================================
# 셀 9: Step 4 - 청킹 전략 비교
# ===============================================
print("\n[Step 4] 청킹 전략 비교")
chunk_strategies = create_korean_optimized_chunks(documents[:10])  # 샘플로 테스트

print("\n청킹 전략 비교 결과:")
for name, chunks in chunk_strategies.items():
    avg_size = sum(len(c.page_content) for c in chunks) / len(chunks) if chunks else 0
    print(f"  {name}: {len(chunks)}개 청크, 평균 {avg_size:.0f}자")

# 최적 전략 선택
best_strategy = "korean_recursive"
print(f"\n선택된 전략: {best_strategy}")


[Step 4] 청킹 전략 비교

텍스트 분할 방법 1: 한국어 최적화 RecursiveCharacterTextSplitter
  - 생성된 청크 수: 22

텍스트 분할 방법 2: 기본 RecursiveCharacterTextSplitter
  - 생성된 청크 수: 21

텍스트 분할 방법 3: CharacterTextSplitter
  - 생성된 청크 수: 10

청킹 전략 비교 결과:
  korean_recursive: 22개 청크, 평균 357자
  basic_recursive: 21개 청크, 평균 394자
  character: 10개 청크, 평균 720자

선택된 전략: korean_recursive


In [9]:
# ===============================================
# 셀 10: Step 5 - 전체 문서 청킹
# ===============================================
print("\n[Step 5] 전체 문서 청킹")
chunk_strategies_full = create_korean_optimized_chunks(documents)
chunks = chunk_strategies_full[best_strategy]

print(f"전체 문서 청킹 완료:")
print(f"  - 총 청크 수: {len(chunks)}")
print(
    f"  - 평균 청크 크기: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f}자"
)


[Step 5] 전체 문서 청킹

텍스트 분할 방법 1: 한국어 최적화 RecursiveCharacterTextSplitter
  - 생성된 청크 수: 1736

텍스트 분할 방법 2: 기본 RecursiveCharacterTextSplitter
  - 생성된 청크 수: 1638

텍스트 분할 방법 3: CharacterTextSplitter
  - 생성된 청크 수: 428
전체 문서 청킹 완료:
  - 총 청크 수: 1736
  - 평균 청크 크기: 373자


In [10]:
# ===============================================
# 셀 11: Step 6 - 메타데이터 추가
# ===============================================
print("\n[Step 6] 메타데이터 추가")
chunks = add_medical_metadata(chunks)

# 섹션 분포 확인
section_distribution = {}
for chunk in chunks:
    section = chunk.metadata.get("section_kr", "일반")
    section_distribution[section] = section_distribution.get(section, 0) + 1

print("섹션별 청크 분포:")
for section, count in sorted(
    section_distribution.items(), key=lambda x: x[1], reverse=True
):
    percentage = (count / len(chunks)) * 100
    print(f"  {section}: {count}개 ({percentage:.1f}%)")


[Step 6] 메타데이터 추가

메타데이터 추가 중...
섹션별 청크 분포:
  일반: 999개 (57.5%)
  치료: 287개 (16.5%)
  진단: 279개 (16.1%)
  운동요법: 82개 (4.7%)
  식이요법: 48개 (2.8%)
  합병증: 41개 (2.4%)


In [11]:
# ===============================================
# 셀 12: Step 7 - 벡터 DB 생성
# ===============================================
import shutil
import os

print("\n[Step 7] 벡터 DB 생성")

# 기존 벡터 DB 디렉토리 완전 삭제
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"기존 벡터 DB 삭제 완료: {PERSIST_DIRECTORY}")

# 새로운 벡터 DB 생성
print("\n[Step 7] 벡터 DB 생성 (새로 시작)")
vectorstore = create_vectorstore_safe(
    chunks, embeddings, PERSIST_DIRECTORY, batch_size=50
)

# 임베딩 차원 확인
test_embedding = embeddings.embed_query("테스트")
print(f"현재 임베딩 차원: {len(test_embedding)}")

# 벡터 DB 생성
try:
    vectorstore = create_vectorstore_safe(
        chunks, embeddings, PERSIST_DIRECTORY, batch_size=50
    )
    print(f"벡터 DB 생성 완료")
    print(f"  - 저장 위치: {PERSIST_DIRECTORY}")
    print(f"  - 저장된 문서 수: {len(chunks)}")
except Exception as e:
    print(f"벡터 DB 생성 실패: {e}")
    raise


[Step 7] 벡터 DB 생성
기존 벡터 DB 삭제 완료: D:\data\chroma_db

[Step 7] 벡터 DB 생성 (새로 시작)

벡터 DB 생성 중 (총 1736개 청크)...
청크가 많아 배치 처리를 사용합니다 (배치 크기: 50)
  처리 중: 1-50/1736
  처리 중: 51-100/1736
  처리 중: 101-150/1736
  처리 중: 151-200/1736
  처리 중: 201-250/1736
  처리 중: 251-300/1736
  처리 중: 301-350/1736
  처리 중: 351-400/1736
  처리 중: 401-450/1736
  처리 중: 451-500/1736
  처리 중: 501-550/1736
  처리 중: 551-600/1736
  처리 중: 601-650/1736
  처리 중: 651-700/1736
  처리 중: 701-750/1736
  처리 중: 751-800/1736
  처리 중: 801-850/1736
  처리 중: 851-900/1736
  처리 중: 901-950/1736
  처리 중: 951-1000/1736
  처리 중: 1001-1050/1736
  처리 중: 1051-1100/1736
  처리 중: 1101-1150/1736
  처리 중: 1151-1200/1736
  처리 중: 1201-1250/1736
  처리 중: 1251-1300/1736
  처리 중: 1301-1350/1736
  처리 중: 1351-1400/1736
  처리 중: 1401-1450/1736
  처리 중: 1451-1500/1736
  처리 중: 1501-1550/1736
  처리 중: 1551-1600/1736
  처리 중: 1601-1650/1736
  처리 중: 1651-1700/1736
  처리 중: 1701-1736/1736
벡터 DB 저장 완료
현재 임베딩 차원: 768

벡터 DB 생성 중 (총 1736개 청크)...
청크가 많아 배치 처리를 사용합니다 (배치 크기: 50)
  처리 중: 1-5

In [12]:
# ===============================================
# 셀 13: Step 8 - 검색 품질 평가
# ===============================================
print("\n[Step 8] 검색 품질 평가")
search_results = evaluate_search_quality(vectorstore, test_queries)

# 평균 점수 계산
if search_results:
    avg_score = sum(r["top_score"] for r in search_results if r["top_score"]) / len(
        search_results
    )
    print(f"\n전체 평균 거리 점수: {avg_score:.3f}")
    print(interpret_chroma_score(avg_score))


[Step 8] 검색 품질 평가

검색 품질 평가

질문: 이 문서의 주요 내용은 무엇인가요?
--------------------------------------------------

  결과 1:

    거리 점수: 0.495
    유사도: 75.2%
    품질: 좋음
    
    섹션: 합병증
    내용: 함. - 각 핵심질문은 2022년 진료지침에 사용했던 검색어, 검색식을 업데이트 및 보강함. - 29개 세부 소주제의 검색어, 검색식 및 검색결과는 파일 형태로 보관함. 2) 근거(진료지침)의 검색 ● 근거문헌에 대해 집필그룹의 구성원이 문헌평가를 할 때 다음과 같은...

  결과 2:

    거리 점수: 0.495
    유사도: 75.2%
    품질: 좋음
    
    섹션: 합병증
    내용: 함. - 각 핵심질문은 2022년 진료지침에 사용했던 검색어, 검색식을 업데이트 및 보강함. - 29개 세부 소주제의 검색어, 검색식 및 검색결과는 파일 형태로 보관함. 2) 근거(진료지침)의 검색 ● 근거문헌에 대해 집필그룹의 구성원이 문헌평가를 할 때 다음과 같은...

  결과 3:

    거리 점수: 0.525
    유사도: 73.7%
    품질: 보통
    
    섹션: 일반
    내용: . - 진료지침관련 색인단어는 다음의 조합으로 검색하였음 (Guideline* or Practice guideline* or Clinical practice guideline* or Recommendation* or Consensus) - 문헌검색은 전문사서 前) 제일...

  [MMR 검색 - 다양성 확보]
    1. [합병증] 함. - 각 핵심질문은 2022년 진료지침에 사용했던 검색어, 검색식을 업데이트 및 보강함. - 29개 세부 소주제의 검색어, 검색식 및 검색결...
    2. [일반] ) 1.01 (0.84–1.21) 1.06 (0.82–1.37) NA NA NA NA NA NA All-cause mortal

In [13]:
# ===============================================
# 셀 14: Step 9 - 실험 결과 저장
# ===============================================
print("\n[Step 9] 실험 결과 저장")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_results = {
    "experiment_info": {
        "timestamp": timestamp,
        "notebook": "02_RAG_01_Data indexing_v2.ipynb",
        "purpose": "2주차 과제 - 한국어 의료 PDF RAG 시스템",
    },
    "data_statistics": {
        "total_pages": len(documents),
        "total_chunks": len(chunks),
        "avg_chunk_size": sum(len(c.page_content) for c in chunks) / len(chunks),
    },
    "chunking_strategy": best_strategy,
    "embedding_model": EMBEDDING_TYPE,
    "section_distribution": section_distribution,
    "vector_db": {"type": "Chroma", "persist_directory": PERSIST_DIRECTORY},
}

output_path = Path(PDF_PATH) / f"rag_experiment_{timestamp}.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(experiment_results, f, ensure_ascii=False, indent=2)

print(f"실험 결과 저장 완료: {output_path}")


[Step 9] 실험 결과 저장
실험 결과 저장 완료: D:\data\rag_experiment_20251201_175820.json


In [14]:
# ===============================================
# 셀 16: 추가 검색 테스트 함수
# ===============================================
def search_and_interpret(query):
    """검색 실행 및 결과 해석"""
    results = vectorstore.similarity_search_with_score(query, k=3)

    print(f"\n질문: {query}")
    print("=" * 50)

    for i, (doc, score) in enumerate(results, 1):
        print(f"\n결과 {i}:")
        print(interpret_chroma_score(score))
        print(f"섹션: {doc.metadata.get('section_kr', 'N/A')}")
        print(f"내용: {doc.page_content[:200]}...")

    return results


# 테스트 예시
print("추가 검색 테스트 함수 준비 완료")
print("사용법: search_and_interpret('검색할 내용')")

추가 검색 테스트 함수 준비 완료
사용법: search_and_interpret('검색할 내용')
